In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import h5py as h5
import ytree
import yt
import cPickle as pickle
import os
import math

In [2]:
from matplotlib import rc, rcParams
rc('font', size=18)
rc('xtick', direction='in')
rc('ytick', direction='in')

# Read halo merger trees

In [3]:
trees = ytree.load('rockstar_halos/trees/tree_0_0_0.dat')
trees.add_alias_field('mass', 'Mvir')

ytree: [WARNING  ] 2018-05-15 09:11:19,655 Overriding field "mass" that already exists as alias field.


Performance will be improved by saving this arbor with "save_arbor" and reloading:
	>>> a = ytree.load("rockstar_halos/trees/tree_0_0_0.dat")
	>>> fn = a.save_arbor()
	>>> a = ytree.load(fn)


In [4]:
mrange = [1e8, 1e10]
halo_mmp = []
for tree in trees:
    if tree['mass'].in_units('Msun') < mrange[0] or tree['mass'].in_units('Msun') > mrange[1]:
        continue
    mmp = dict(center=[], mass=[], redshift=[], rvir=[])
    for prog in tree.pwalk():
        pos = yt.YTArray([prog['x'], prog['y'], prog['z']]) / trees.box_size
        mmp['center'].append(pos)
        mmp['mass'].append(prog['mass'].in_units('Msun'))
        mmp['redshift'].append(prog['redshift'])
        mmp['rvir'].append(prog['virial_radius'] / trees.box_size)
    mmp['center'] = yt.YTArray(mmp['center'])
    mmp['mass'] = yt.YTArray(mmp['mass'])
    mmp['redshift'] = yt.YTArray(mmp['redshift'])
    mmp['rvir'] = yt.YTArray(mmp['rvir'])
    halo_mmp.append(mmp)

Loading tree roots: 100%|██████████| 334185987/334185987 [00:00<00:00, 388712224.13it/s]


In [5]:
print halo_mmp[0]['rvir']

[0.00099201 0.00095577 0.00092356 0.00090081 0.00088599 0.00087553
 0.0008637  0.00085342 0.00084155 0.00083225 0.0008219  0.00081377
 0.00080669 0.00077102 0.00078919 0.00077972 0.00076856 0.00075852
 0.00074708 0.00073754 0.00072352 0.00070454 0.00069736 0.00062662
 0.00058606 0.00066201 0.00064563 0.00062384 0.00061317 0.00060021
 0.00058665 0.00057243 0.00056852 0.00055447 0.00054725 0.00054165
 0.00053581 0.00052831 0.00052088 0.00051511 0.00050595 0.00041824
 0.00042151 0.00042655 0.00043148 0.00043627 0.00045887 0.00044687
 0.00043419 0.00043563 0.00042609 0.00034764 0.00039356 0.00035521
 0.00032651 0.00031246 0.00029701 0.00031454 0.00030937 0.00030004
 0.00027137 0.00029225 0.00028602 0.00027827 0.00027444 0.00026606
 0.00026243 0.0002594  0.00025447 0.00025074 0.00024419 0.00024144
 0.00023623 0.00023056 0.00022736 0.00022275 0.00021739 0.00021271
 0.00020461 0.00019581 0.00018345 0.00017715 0.00016373 0.00015127
 0.00014327 0.00013398 0.00012866 0.00011796 0.00011095 0.0001

# Read simulation redshifts

In [6]:
pfn = []
redshifts = []
if os.path.exists('redshifts.dat'):
    lines = open('redshifts.dat').readlines()
    for l in lines:
        pfn.append(l.split(':')[0])
        redshifts.append(float(l.split('=')[-1]))
else:
    ts = yt.DatasetSeries('DD????/output_????')
    for ds in ts:
        redshifts.append(ds.current_redshift)
        pfn.append(ds.parameter_filename)
    
redshifts = np.array(redshifts)
pfn = np.array(pfn)
isort = redshifts.argsort().astype('int')[::-1]
redshifts = redshifts[isort]
pfn = pfn[isort]

# Read halo properties

In [ ]:
print halo_mmp[0]['rvir']

[0.00099201 0.00095577 0.00092356 0.00090081 0.00088599 0.00087553
 0.0008637  0.00085342 0.00084155 0.00083225 0.0008219  0.00081377
 0.00080669 0.00077102 0.00078919 0.00077972 0.00076856 0.00075852
 0.00074708 0.00073754 0.00072352 0.00070454 0.00069736 0.00062662
 0.00058606 0.00066201 0.00064563 0.00062384 0.00061317 0.00060021
 0.00058665 0.00057243 0.00056852 0.00055447 0.00054725 0.00054165
 0.00053581 0.00052831 0.00052088 0.00051511 0.00050595 0.00041824
 0.00042151 0.00042655 0.00043148 0.00043627 0.00045887 0.00044687
 0.00043419 0.00043563 0.00042609 0.00034764 0.00039356 0.00035521
 0.00032651 0.00031246 0.00029701 0.00031454 0.00030937 0.00030004
 0.00027137 0.00029225 0.00028602 0.00027827 0.00027444 0.00026606
 0.00026243 0.0002594  0.00025447 0.00025074 0.00024419 0.00024144
 0.00023623 0.00023056 0.00022736 0.00022275 0.00021739 0.00021271
 0.00020461 0.00019581 0.00018345 0.00017715 0.00016373 0.00015127
 0.00014327 0.00013398 0.00012866 0.00011796 0.00011095 0.0001

In [ ]:
pfile = 'fesc.cpkl'
if os.path.exists(pfile):
    fp = open(pfile, 'rb')
    catalog = pickle.load(fp)
    fp.close()
else:
    catalog = []
    for fn in pfn:
        entry = {}
        fp = h5.File('fesc-data/p%s_fesc_halos.h5' % (fn.split('/')[1]), 'r')
        nhalos = len(fp.keys())
        if nhalos == 0:
            catalog.append(entry)
            fp.close()
            continue
        gp0 = fp.values()[0]
        avoid = ['Pop2_Stars', 'Pop3_Stars', 'fesc_bins', 'fesc_hist']
        vector_fields = ['center']
        # Create empty entry
        for f in gp0:
            if f not in avoid:
                if f in vector_fields:
                    entry[f] = np.zeros((nhalos,3))
                else:
                    entry[f] = np.zeros(nhalos)
        # Read the data
        i = 0
        for g in fp.itervalues():
            for f in g:
                if f not in avoid:
                    entry[f][i] = g[f].value
            i += 1
        catalog.append(entry)
        fp.close()
    # Serialize properties
    fp = open(pfile, 'wb')
    pickle.dump(catalog, fp)
    fp.close()

# Link halo properties to merger tree nodes

In [ ]:
def find_nearest(array, value):
    return np.abs(array - value).argmin()

In [ ]:
print halo_mmp[0]['rvir']

In [ ]:
fields = catalog[-1].keys()
fields.remove('center')
fields.remove('rvir')
for halo in halo_mmp:
    nnode = len(halo['redshift'])
    for f in fields:
        halo[f] = np.zeros(nnode)
    for inode in range(nnode):
        ip = find_nearest(redshifts, halo['redshift'][inode])
        
        # dr = separation, R = radius of target halo, r = radii of all catalog halos
        dr = np.sqrt(((halo['center'][inode] - catalog[ip]['center'])**2).sum(1))
        R = halo['rvir'][inode]
        r = catalog[ip]['rvir'] * (1 + redshifts[ip]) / trees.box_size.in_units('kpc').v
        
        # Calculate the sphere-sphere intersection. The maximum is the match.
        # http://mathworld.wolfram.com/Sphere-SphereIntersection.html
        vol = np.pi * (R+r-dr)**2 * (dr**2 + 2*dr*r - 3*r**2 + 2*dr*R + 6*r*R - 3*R**2) / (12*dr)
        vol[dr > r+R] = 0.0  # No intersection
        vol[dr < R-r] = (4*np.pi/3) * r[dr < R-r]**3  # Complete intersection
        
        match = vol.argmax()
        rvir = catalog[ip]['rvir'][match] * (1 + redshifts[ip]) / trees.box_size.in_units('kpc').v
        # Check whether separation is within the virial radius
        if dr[match] >= rvir:
            continue
        # Add halo properties to mmp history
        for f in fields:
            halo[f][inode] = catalog[ip][f][match]
    # Calculate SFR (last 20 Myr) [Msun/yr]
    halo['SFR'] = halo['Mstar_pop2_young'] / 20e6
    halo['fesc'] = np.maximum(halo['fesc'], 1e-6)

In [ ]:
print halo['mass']

# Save to file for plotting

In [ ]:
fp = h5.File('catalog.h5', 'w')
for i,halo in enumerate(halo_mmp):
    hname = "Halo%8.8d" % (i)
    gp = fp.create_group(hname)
    for k,v in halo.items():
        gp[k] = v
fp.close()

In [ ]:
1 / (halo_mmp[0]['redshift']+1)

In [ ]:
halo_mmp[0]['mass']

In [ ]:
len(halo_mmp)

In [ ]:
fields = ['mass', 'Mstar', 'SFR', 'fesc', 'fgas', 'fstar']
log = [True, True, True, True, False, True]
ylabel = [r'M$_{\rm vir}$ [M$_\odot$]', r'M$_{\star}$ [M$_\odot$]', 'SFR [M$_\odot$/yr]',
          r'f$_{\rm esc}$', r'f$_{\rm gas}$', r'f$_\star$']
nhalos = len(halo_mmp)
for i in range(nhalos):
    if halo_mmp[i]['Mstar'][0] < 1: continue
    fig, ax = plt.subplots(6, 1, figsize=(8,16), sharex=True)
    plt.subplots_adjust(hspace=1e-3)
    for j in range(len(fields)):
        if log[j]:
            ax[j].semilogy(halo_mmp[i]['redshift'], halo_mmp[i][fields[j]], lw=3)
        else:
            ax[j].plot(halo_mmp[i]['redshift'], halo_mmp[i][fields[j]], lw=3)
        ax[j].set_ylabel(ylabel[j])
    ax[0].set_title('Halo %d' % (i))
    ax[-1].set_xlabel('Redshift')
    plt.savefig('halo%6.6d-evo.png' % (i))